In [1]:
import pandas as pd
import sqlite3

In [2]:
# Establish a connection to the SQLite database
conn = sqlite3.connect('FinalDB.db')

# Verify the connection
if conn:
    print("Connection established successfully!")
else:
    print("Connection failed.")


Connection established successfully!


In [4]:
%load_ext sql

# Connect to the SQLite database
%sql sqlite:///FinalDB.db

In [5]:
# URLs to the datasets
census_data_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'
chicago_public_schools_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'
chicago_crime_data_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01'

# Load the data into pandas dataframes
census_data = pd.read_csv(census_data_url)
chicago_public_schools = pd.read_csv(chicago_public_schools_url)
chicago_crime_data = pd.read_csv(chicago_crime_data_url)

# Establish a connection to the SQLite database
conn = sqlite3.connect('FinalDB.db')

# Load the dataframes into the database as tables
census_data.to_sql('CENSUS_DATA', conn, if_exists='replace', index=False)
chicago_public_schools.to_sql('CHICAGO_PUBLIC_SCHOOLS', conn, if_exists='replace', index=False)
chicago_crime_data.to_sql('CHICAGO_CRIME_DATA', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print("Data has been loaded into the database successfully.")


Data has been loaded into the database successfully.


In [ ]:
#Find the total number of crimes recorded in the CRIME table.

In [6]:

%%sql
select count(*) from chicago_crime_data;

 * sqlite:///FinalDB.db
Done.


count(*)
533


In [ ]:
#List community areas with per capita income less than 11000

In [7]:
%%sql
select COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, PER_CAPITA_INCOME 
from CENSUS_DATA 
where PER_CAPITA_INCOME < 11000

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER,COMMUNITY_AREA_NAME,PER_CAPITA_INCOME
26.0,West Garfield Park,10934
30.0,South Lawndale,10402
37.0,Fuller Park,10432
54.0,Riverdale,8201


In [ ]:
#List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [8]:
%%sql
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION
FROM CHICAGO_CRIME_DATA 
WHERE DESCRIPTION LIKE '%MINOR%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HL266884,LIQUOR LAW VIOLATION,SELL/GIVE/DEL LIQUOR TO MINOR
HK238408,LIQUOR LAW VIOLATION,ILLEGAL CONSUMPTION BY MINOR


In [9]:
#List all kidnapping crimes involving a child?

In [10]:
%%sql
SELECT CASE_NUMBER, PRIMARY_TYPE, DESCRIPTION
FROM CHICAGO_CRIME_DATA
WHERE PRIMARY_TYPE = 'KIDNAPPING' AND DESCRIPTION LIKE '%CHILD%'

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


In [ ]:
#List the kind of crimes that were recorded at schools. (No repetitions)

In [11]:
%%sql
SELECT DISTINCT PRIMARY_TYPE
FROM CHICAGO_CRIME_DATA
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%'

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


In [ ]:
#List the type of schools along with the average safety score for each type

In [12]:
%%sql
SELECT "Elementary, Middle, or High School" AS School_Type, AVG(SAFETY_SCORE) AS Average_Safety_Score
FROM CHICAGO_PUBLIC_SCHOOLS
GROUP BY "Elementary, Middle, or High School"

 * sqlite:///FinalDB.db
Done.


School_Type,Average_Safety_Score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


In [ ]:
#List 5 community areas with highest % of households below poverty line

In [13]:
%%sql
SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM CENSUS_DATA
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


In [ ]:
#Which community area is most crime prone?

In [14]:
%%sql
SELECT COMMUNITY_AREA_NUMBER
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


In [ ]:
#Use a sub-query to find the name of the community area with highest hardship index

In [15]:
%%sql
SELECT COMMUNITY_AREA_NAME
FROM CENSUS_DATA
WHERE HARDSHIP_INDEX = (
    SELECT MAX(HARDSHIP_INDEX)
    FROM CENSUS_DATA
)

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


In [ ]:
#Use a sub-query to determine the Community Area Name with most number of crimes?

In [16]:
%%sql
SELECT COMMUNITY_AREA_NUMBER
FROM CHICAGO_CRIME_DATA
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1


 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0
